In [1]:
from pyiron_atomistics import Project
from pyscal_rdf import StructureGraph
from rdfjobs.lammps.lammps import RDFLammps
from tqdm.notebook import trange

In [2]:
import numpy as np

In [3]:
pr = Project("prod_data_7h")

In [4]:
s = StructureGraph(store=pr)

In [5]:
elements = ["Fe", "Ni", "Cu", "Al", "Ti", "Mg", "Zn"]
#elements = ["Fe"]
reps = 2

In [6]:
for x in trange(len(elements)):
    element = elements[x]
    #create structure
    struct = s.create_element(element, repetitions=(reps, reps, reps))
    #minimise this
    job = pr.create_job(job_type=RDFLammps, 
                        job_name=f'{element}_j_1', 
                        delete_existing_job=True)
    job.structure = struct
    if element in ["Mg", "Zn"]:
        job.potential = job.list_potentials()[1]
    else:
        job.potential = job.list_potentials()[0]
    job.calc_minimize()
    job.run()
    energy_bulk = job.output.energy_tot[-1]
    number_atoms = struct.natoms

    #collect the minimised structure and add it to graph
    min_struct = job.get_structure_as_system()
    s.add_structure_to_graph(min_struct)
    #delete one atom
    del min_struct[np.random.randint(0, min_struct.natoms)]

    #now minimise this structure
    job = pr.create_job(job_type=RDFLammps, 
                        job_name=f'{element}_j_2', 
                        delete_existing_job=True)
    job.structure = min_struct
    if element in ["Mg", "Zn"]:
        job.potential = job.list_potentials()[1]
    else:
        job.potential = job.list_potentials()[0]
    job.calc_minimize()
    job.run()
    energy_vac = job.output.energy_tot[-1]

    #now calculate vacancy formation energy
    form_energy = energy_vac - ((number_atoms-1)/number_atoms)*energy_bulk
    #add it to the graph
    s.add_calculated_quantity("VacancyFormationEnergy", 
                              form_energy, 
                              unit="EV",
                              sample=job._final_sample)

  0%|          | 0/7 [00:00<?, ?it/s]

Fe
The job Fe_j_1 was saved and received the ID: 697
The job Fe_j_2 was saved and received the ID: 698
Ni
The job Ni_j_1 was saved and received the ID: 699
The job Ni_j_2 was saved and received the ID: 700
Cu
The job Cu_j_1 was saved and received the ID: 701
The job Cu_j_2 was saved and received the ID: 702
Al
The job Al_j_1 was saved and received the ID: 703
The job Al_j_2 was saved and received the ID: 704
Ti
The job Ti_j_1 was saved and received the ID: 705
The job Ti_j_2 was saved and received the ID: 706
Mg
The job Mg_j_1 was saved and received the ID: 707
The job Mg_j_2 was saved and received the ID: 708
Zn
The job Zn_j_1 was saved and received the ID: 709
The job Zn_j_2 was saved and received the ID: 710


In [7]:
form_energy, s.n_samples

(0.15976862346362708, 28)